# Running the TTGammaProcessor

This cell will copy the test files from their location on eos to your local area. This only needs to be done once!

In [4]:
# #If you have not already done so, you can copy the files to test the code on from here
# # ONLY NEEDS TO BE DONE ONCE, CAN BE COMMENTED OUT WHEN YOU 
 !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTGamma_1l.root .
 !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/TTbar_1l.root .
 !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/WGamma.root .
 !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/ZGamma.root .
 !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/W4Jets.root .
 !xrdcp root://cmseos.fnal.gov//store/user/cmsdas/2021/long_exercises/TTGamma/TestFiles/ZJets.root .

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3006] File exists

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3006] File exists

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3006] File exists

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3006] File exists

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3006] File exists

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3006] File exists



In [1]:
%load_ext autoreload
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

List of samples to be run on (fileset variable) and a dictionary containing the number of events processed for each sample

In [2]:
fileset = {'TTGamma_SingleLept':['TTGamma_1l.root'],
           'TTbarPowheg_Semilept':['TTbar_1l.root'],
           'W4jets':['W4Jets.root'],
           'WGamma_01J_5f':['WGamma.root'],
           'ZGamma_01J_5f_lowMass':['ZGamma.root'],
           'DYjetsM50':['ZJets.root'],
        }



Run the TTGammaProcessor on the list of files included in fileset.

You can specify the chunksize and maximum number of chunks to process from each sample (selecting a small number of events and one chunk will force coffea to process only a subset of the events for quicker debugging)

In [3]:
#autoreload forces the kernel to reload the processor to include any new changes
%autoreload 2
from ttgamma import TTGammaProcessor
import awkward as ak

import time
tstart = time.time()

#Run Coffea code using uproot
output = processor.run_uproot_job(
    fileset,
    "Events",
    TTGammaProcessor(isMC=True),
    processor.iterative_executor,
    executor_args={'schema': NanoAODSchema,'workers': 4},
    chunksize=1000,
    maxchunks=1,
)

elapsed = time.time() - tstart
print("Total time: %.1f seconds"%elapsed)
print("Total rate: %.1f events / second"%(output['EventCount'].value/elapsed))

Processing:   0%|          | 0/6 [00:00<?, ?chunk/s]

/uscms/home/jhossain/nobackup/gitexer/TTGamma_LongExercise/ttgenv/lib/python3.6/site-packages/coffea/util.py:91: FutureWarning: In coffea version 0.8, this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
str: This utility has moved to the `coffea.analysis_tools` subpackage and has new features, check it out!
  warnings.warn(message, FutureWarning)


Total time: 8.6 seconds
Total rate: 698.8 events / second


In [19]:
output

{'all_photon_pt': <Hist (dataset,pt) instance at 0x7f6db9699f60>,
 'EventCount': value_accumulator(int, 6008)}

In [22]:
from coffea import hist
hist.plot1d(output['pho_pt'],overlay='dataset')

KeyError: 'pho_pt'

# Accessing Arrays Interactively

Below is an example of loading a NanoAOD file interactively. This can be very useful for developing the code, and debugging any issues. Use this area to build your intuition for working with Coffea and awkward arrays!

In [23]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

fname = "./TTGamma_1l.root"
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()

Once you have opened the file, you can explore its contents using the 'fields' syntax

In [24]:
events.fields

['run',
 'GenMET',
 'Flag',
 'HLT',
 'GenJet',
 'LHEPdfWeight',
 'Pileup',
 'GenPart',
 'MET',
 'Jet',
 'LHEScaleWeight',
 'event',
 'Electron',
 'PSWeight',
 'Muon',
 'luminosityBlock',
 'fixedGridRhoFastjetAll',
 'Generator',
 'Photon',
 'LHEWeight']

In [25]:
events.GenPart.fields

['eta',
 'mass',
 'phi',
 'pt',
 'genPartIdxMother',
 'pdgId',
 'status',
 'statusFlags',
 'genPartIdxMotherG',
 'distinctParentIdxG',
 'childrenIdxG',
 'distinctChildrenIdxG']

There is also a docstring for each of these variables in NanoAOD, which you can access using '?':

In [26]:
events.Jet.rawFactor?

In [17]:
triJetPt = (triJet.first + triJet.second + triJet.third).pt
print(triJetPt)

NameError: name 'triJet' is not defined